In [14]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(10)  # Espera implícita global de 10 segundos
    return driver

def fetch_mtga_data(url):
    driver = setup_driver()
    driver.get(url)

    # Usar texto visível para localizar o botão "Copy to MTGA"
    try:
        copy_button = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[text()='Copy to MTGA']"))
        )
        print("Botão 'Copy to MTGA' encontrado.")
        
        # Capturar o botão "Copy to MTGA" e clicar nele, se necessário
        copy_button_text = copy_button.text
        print(f"Texto do botão: {copy_button_text}")
        
        # Se quiser clicar no botão (não obrigatório)
        # copy_button.click()
        
    except Exception as e:
        print(f"Erro ao localizar o botão 'Copy to MTGA': {e}")

    # Usar texto visível para localizar o link "View Deck"
    try:
        view_deck_link = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//span[text()='View Deck']"))
        )
        print("Link 'View Deck' encontrado.")
        
        # Capturar o texto do botão "View Deck"
        view_deck_text = view_deck_link.text
        print(f"Texto do link: {view_deck_text}")
        
        # Se quiser clicar no link (não obrigatório)
        # view_deck_link.click()
        
    except Exception as e:
        print(f"Erro ao localizar o link 'View Deck': {e}")

    driver.quit()

# URL da página alvo
url = "https://mtga.untapped.gg/meta/decks?meta=previous"
fetch_mtga_data(url)

Botão 'Copy to MTGA' encontrado.
Texto do botão: Copy to MTGA
Link 'View Deck' encontrado.
Texto do link: 


In [27]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains
import os
import time

def setup_driver():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")
    
    # Desativar pop-up de notificações e permissões de clipboard
    prefs = {
        "profile.default_content_setting_values.notifications": 2,  # Bloqueia notificações
        "profile.default_content_setting_values.clipboard": 2       # Bloqueia acesso à área de transferência
    }
    chrome_options.add_experimental_option("prefs", prefs)
    
    service = Service(ChromeDriverManager().install())
    driver = webdriver.Chrome(service=service, options=chrome_options)
    driver.implicitly_wait(10)
    return driver

def close_ads(driver):
    try:
        close_button = driver.find_element(By.XPATH, "//div[@class='PageContainerWithAds__BottomAd-sc-ovl74r-3 dxBbFg']")
        driver.execute_script("arguments[0].remove();", close_button)
        print("Anúncio fechado.")
    except Exception as e:
        print(f"Não foi possível localizar ou fechar o anúncio: {e}")

def scroll_to_element(driver, element):
    driver.execute_script("arguments[0].scrollIntoView();", element)
    time.sleep(1)  # Pequena pausa para garantir que a rolagem seja concluída

def fetch_mtga_data(url):
    driver = setup_driver()
    driver.get(url)
    time.sleep(5)

    close_ads(driver)

    output_dir = 'data\\deck_samples'
    os.makedirs(output_dir, exist_ok=True)

    try:
        # Rolagem gradual e interação com os botões
        idx = 1
        while True:
            try:
                # Tentar localizar o próximo botão "Copy to MTGA"
                button = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, f"(//span[text()='Copy to MTGA'])[{idx}]"))
                )

                # Garantir que o botão está visível
                scroll_to_element(driver, button)

                # Tentar clicar no botão usando ActionChains
                actions = ActionChains(driver)
                actions.move_to_element(button).click().perform()
                time.sleep(2)  # Esperar a execução do clique

                # Capturar o texto do deck
                deck_text_element = WebDriverWait(driver, 10).until(
                    EC.presence_of_element_located((By.XPATH, "//textarea[@class='deck-textarea']"))
                )
                deck_text = deck_text_element.get_attribute("value")
                
                if deck_text:
                    filename = f'deck_{str(idx).zfill(2)}.txt'
                    file_path = os.path.join(output_dir, filename)
                    
                    with open(file_path, 'w', encoding='utf-8') as file:
                        file.write(deck_text + '\n')
                    
                    print(f"Deck {idx} salvo em {filename}.")
                
                idx += 1  # Passar para o próximo botão
            except Exception as e:
                print(f"Erro ao processar o botão {idx}: {e}")
                break  # Parar o loop se não conseguir encontrar ou processar o próximo botão
    except Exception as e:
        print(f"Erro geral ao processar os botões: {e}")

    driver.quit()

# URL da página alvo
url = "https://mtga.untapped.gg/meta/decks?meta=previous"
fetch_mtga_data(url)

Anúncio fechado.
Erro ao processar o botão 1: Message: 
Stacktrace:
	GetHandleVerifier [0x00E78213+26163]
	(No symbol) [0x00E09CC4]
	(No symbol) [0x00D024C3]
	(No symbol) [0x00D47453]
	(No symbol) [0x00D4762B]
	(No symbol) [0x00D86B62]
	(No symbol) [0x00D6AD04]
	(No symbol) [0x00D84661]
	(No symbol) [0x00D6AA56]
	(No symbol) [0x00D3BE89]
	(No symbol) [0x00D3C8CD]
	GetHandleVerifier [0x0114D313+2996019]
	GetHandleVerifier [0x011A1B89+3342249]
	GetHandleVerifier [0x00F07AEF+614159]
	GetHandleVerifier [0x00F0F17C+644508]
	(No symbol) [0x00E127FD]
	(No symbol) [0x00E0F6F8]
	(No symbol) [0x00E0F895]
	(No symbol) [0x00E01C16]
	BaseThreadInitThunk [0x76FC7BA9+25]
	RtlInitializeExceptionChain [0x77A2C10B+107]
	RtlClearBits [0x77A2C08F+191]



In [28]:
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

class TestUntitled():
    def setup_method(self, method):
        self.driver = webdriver.Chrome()
        self.vars = {}
    
    def teardown_method(self, method):
        self.driver.quit()
    
    def test_untitled(self):
        self.driver.get("https://mtga.untapped.gg/meta/decks?meta=previous")
        self.driver.set_window_size(1920, 1080)

        # Esperar até que os botões "Copy to MTGA" estejam visíveis
        buttons = WebDriverWait(self.driver, 20).until(
            EC.presence_of_all_elements_located((By.CSS_SELECTOR, ".sc-hKgJUU"))
        )

        for idx, button in enumerate(buttons):
            try:
                # Clicar no botão "Copy to MTGA"
                button.click()
                
                # Aguardar o conteúdo do deck ser copiado para a área de transferência (simulação)
                time.sleep(1)

                # Extraindo o conteúdo e salvando em arquivos separados
                deck_content = self.driver.find_element(By.CSS_SELECTOR, ".decklist").text
                with open(f"deck_{idx+1}.txt", 'w', encoding='utf-8') as file:
                    file.write(deck_content)
                print(f"Deck {idx+1} salvo com sucesso.")
                
            except Exception as e:
                print(f"Erro ao processar o botão {idx+1}: {e}")
            
            # Rolagem para baixo para carregar mais conteúdo, se necessário
            self.driver.execute_script("window.scrollBy(0, 800);")
            time.sleep(2)  # Aguardar o carregamento

        self.driver.quit()

# Executar o teste
test = TestUntitled()
test.setup_method(None)
test.test_untitled()
test.teardown_method(None)

Erro ao processar o botão 1: Message: no such element: Unable to locate element: {"method":"css selector","selector":".decklist"}
  (Session info: chrome=128.0.6613.114); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7C5EDB5D2+29090]
	(No symbol) [0x00007FF7C5E4E689]
	(No symbol) [0x00007FF7C5D0B1CA]
	(No symbol) [0x00007FF7C5D5EFD7]
	(No symbol) [0x00007FF7C5D5F22C]
	(No symbol) [0x00007FF7C5DA97F7]
	(No symbol) [0x00007FF7C5D8672F]
	(No symbol) [0x00007FF7C5DA65D9]
	(No symbol) [0x00007FF7C5D86493]
	(No symbol) [0x00007FF7C5D509B1]
	(No symbol) [0x00007FF7C5D51B11]
	GetHandleVerifier [0x00007FF7C61F8C5D+3295277]
	GetHandleVerifier [0x00007FF7C6244843+3605523]
	GetHandleVerifier [0x00007FF7C623A707+3564247]
	GetHandleVerifier [0x00007FF7C5F96EB6+797318]
	(No symbol) [0x00007FF7C5E5980F]
	(No symbol) [0x00007FF7C5E553F4]
	(No symbol) [0x00007FF7C5E

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=128.0.6613.114)
Stacktrace:
	GetHandleVerifier [0x00007FF7C5EDB5D2+29090]
	(No symbol) [0x00007FF7C5E4E689]
	(No symbol) [0x00007FF7C5D0B1CA]
	(No symbol) [0x00007FF7C5CDFAF5]
	(No symbol) [0x00007FF7C5D8E2E7]
	(No symbol) [0x00007FF7C5DA5EE1]
	(No symbol) [0x00007FF7C5D86493]
	(No symbol) [0x00007FF7C5D509B1]
	(No symbol) [0x00007FF7C5D51B11]
	GetHandleVerifier [0x00007FF7C61F8C5D+3295277]
	GetHandleVerifier [0x00007FF7C6244843+3605523]
	GetHandleVerifier [0x00007FF7C623A707+3564247]
	GetHandleVerifier [0x00007FF7C5F96EB6+797318]
	(No symbol) [0x00007FF7C5E5980F]
	(No symbol) [0x00007FF7C5E553F4]
	(No symbol) [0x00007FF7C5E55580]
	(No symbol) [0x00007FF7C5E44A1F]
	BaseThreadInitThunk [0x00007FF926FA257D+29]
	RtlUserThreadStart [0x00007FF92852AF28+40]


In [ ]:
# TO tomando um pau por conta de autenticação, vou tentar usar API...